In [14]:
%pip install --upgrade --quiet  langchain-core langchain-openai langchain-anthropic langchain_core.auth

ERROR: Could not find a version that satisfies the requirement langchain_core.auth (from versions: none)
ERROR: No matching distribution found for langchain_core.auth
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['X-Api-Key'] = os.getenv("X_API_KEY")

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic} and always add the name of your model at the end of the joke."
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4X")
openai_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

openai_chain.invoke("ice cream")

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4X` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [9]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOllama(model="llama2")
ollama_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

ollama_chain.invoke("ice cream")

'\nWhy did the ice cream cone go to the party?\n\nBecause it was a scream!'

In [14]:
fallback_chain = openai_chain.with_fallbacks([ollama_chain])

fallback_chain.invoke("ice cream")
# await fallback_chain.ainvoke("ice cream")
fallback_chain.batch(["ice cream", "spaghetti", "dumplings"])

['Why did the ice cream go to the party? Because it was feeling a little "sweet"!',
 '\nWhy did the spaghetti go to the party?\n\nBecause it was feeling saucy!',
 '\nWhy did the dumpling go to therapy?\n\nBecause it was feeling a little "stuffed"!']

In [ ]:
%pip install langchain_experimental langchainhub

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

In [5]:
tools = [PythonREPLTool()]

In [6]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [7]:
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [8]:
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)



In [9]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [15]:
agent_executor.invoke({"input": "What is the 13th fibonacci number?"})



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(13)`


The 13th Fibonacci number is 233.

> Finished chain.


{'input': 'What is the 13th fibonacci number?',
 'output': 'The 13th Fibonacci number is 233.'}

In [ ]:
%pip install torch

In [13]:
agent_executor.invoke(
    {
        "input": """Understand, write a single neuron neural network in PyTorch.
Take synthetic data for y=2x. Train for 1000 epochs and print every 100 epochs.
Return prediction for x = 8"""
    }
)



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `import torch
import torch.nn as nn
import torch.optim as optim

# Create synthetic data
x = torch.tensor([[1.0], [2.0], [3.0], [4.0], [5.0], [6.0], [7.0], [8.0]], dtype=torch.float32)
y = torch.tensor([[2.0], [4.0], [6.0], [8.0], [10.0], [12.0], [14.0], [16.0]], dtype=torch.float32)

# Define a single neuron neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear = nn.Linear(1, 1)
    
    def forward(self, x):
        return self.linear(x)

# Instantiate the model
model = NeuralNetwork()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}, Loss:

{'input': 'Understand, write a single neuron neural network in PyTorch.\nTake synthetic data for y=2x. Train for 1000 epochs and print every 100 epochs.\nReturn prediction for x = 8',
 'output': 'The prediction for x=8 using the trained single neuron neural network is approximately 15.99.'}